In [6]:
from __future__ import division
import pandas as pd
import numpy as np
import logging
import math
import time
import statistics as stats
import json
from datetime import datetime, timedelta
import os, sys
import matplotlib.pyplot as plt
import glob

In [7]:
feature_file_path = "features/vag_features.csv"

f_df = pd.read_csv(feature_file_path)
print(f_df.head())

   STATE        FD    FD_EXT    FD_FLX       VMS   VMS_EXT   VMS_FLX  \
0      1  1.466570  1.775843  1.820782  0.025042  0.019168  0.029947   
1      1  1.766607  1.627739  1.922575  0.004414  0.003966  0.003024   
2      1  1.744344  1.869978  2.031547  0.003247  0.001370  0.004936   
3      1  2.287178  2.038849  2.328853  0.000181  0.000233  0.000093   
4      1  1.343740  1.526733  1.595508  0.010186  0.006659  0.003177   

           LI      LI_EXT      LI_FLX  
0  889.089284  614.701096  674.587386  
1  631.959945  454.240634  460.176026  
2  404.492380  217.192115  337.943733  
3  261.686751  252.636066   63.201615  
4  268.935033  199.078012  181.254098  


In [16]:
#!pip install scikit-learn 
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

In [17]:
features = f_df.drop('STATE', axis=1)
labels = f_df['STATE']

In [18]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.5, random_state=42)

auc_values = {}

for feature in features.columns:
    feature_train = X_train[feature]
    feature_test = X_test[feature]
    
    # Compute the AUC for the feature
    auc_value = roc_auc_score(y_test, feature_test)
    auc_values[feature] = auc_value

# Print the AUC values for each feature
for feature, auc in auc_values.items():
    print(f'AUC (Az) Value for {feature}: {auc:.4f}')

AUC (Az) Value for FD: 0.6573
AUC (Az) Value for FD_EXT: 0.6142
AUC (Az) Value for FD_FLX: 0.7478
AUC (Az) Value for VMS: 0.2543
AUC (Az) Value for VMS_EXT: 0.2996
AUC (Az) Value for VMS_FLX: 0.1983
AUC (Az) Value for LI: 0.4978
AUC (Az) Value for LI_EXT: 0.5000
AUC (Az) Value for LI_FLX: 0.4784
